In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# This notebook is used to process the data of H&M RS

## 1.clean data in article.csv

In [ ]:
#read article.csv file
article = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
print(article.shape)
article.head(5)

## remove name feature only left code feature

In [ ]:
article.columns

In [ ]:
code_feature = ['article_id', 'product_code', 'product_type_no',
                'product_group_name', 'graphical_appearance_no',
                'colour_group_code', 'perceived_colour_value_id', 
                'perceived_colour_master_id', 
                'department_no', 'index_code',
                'index_group_no', 'section_no',
                'garment_group_no']
text_des = ['detail_desc']

# change string into code
from sklearn.preprocessing import LabelEncoder
change_column=['product_group_name', 'index_code']
for column in change_column:
    lbe = LabelEncoder()
    lbe.fit(article[column].unique())
    article[column] = lbe.transform(article[column])

#for nn.embedding
article_code = article[code_feature]
article_code.head()

#for BERT embed
article_describe = article[text_des]

In [ ]:
article_code.isnull().sum()

In [ ]:
article_code.head()

In [ ]:
article_describe.head()

In [ ]:
def build_map(data,col_name):
    key = data[col_name].unique()
    m = dict(zip(key, range(len(key))))
    data[col_name] = data[col_name].map(lambda x: m[x])
    return key, m

_, article_map = build_map(article_code,'article_id')
_, tmp = build_map(article_code,'product_type_no')

In [ ]:
import pickle
if not os.path.exists('process_0'):
    os.mkdir('process_0')
pd.to_pickle(article_code,'process_0/article_code.pkl')
pd.to_pickle(article_describe,'process_0/article_describe.pkl')
with open('process_0/atricle_map.pkl','wb') as f:
    pickle.dump(article_map,f)

''' # read article_map
with open(''process_0/atricle_map.pkl','rb') as f:
    article_map = pickle.load(f)
'''

## article.csv process done!

## 2.clean data in customers.csv

In [ ]:
# read customers.csv
customers = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/customers.csv')
print('customers shape:',customers.shape)
customers.head()

In [ ]:
customers.isnull().sum()
#there are much missing values in customers.csv, but some can be filled like 'club_member_status'

In [ ]:
customers.describe()
#FN and Active seems only have 1.0 value, may be we can fill 0 into missing value.

In [ ]:
customers.fashion_news_frequency.unique()

In [ ]:
customers.club_member_status.unique()

In [ ]:
customer_column_name = ['customer_id', 'FN', 'Active', 'club_member_status',
                        'fashion_news_frequency', 'age', 'postal_code']

#fill all missing value
customers.club_member_status[customers.club_member_status.isnull()] = 'PRE-CREATE'
customers.fashion_news_frequency[customers.fashion_news_frequency.isnull()] = 'NONE'
customers.fashion_news_frequency[customers.fashion_news_frequency =='None'] = 'NONE'
customers.FN[customers.FN.isnull()] = 0.
customers.Active[customers.Active.isnull()] = 0.
customers.age[customers.age.isnull()] = customers.age.mode().values[0]

In [ ]:
from sklearn.preprocessing import LabelEncoder
lbe = LabelEncoder()
lbe.fit(customers.postal_code)
customers.postal_code = lbe.transform(customers.postal_code)


In [ ]:
customers.head()

In [ ]:
_, customer_map = build_map(customers, 'customer_id')

In [ ]:
if not os.path.exists('process_0'):
    os.mkdir('process_0')
pd.to_pickle(customers,'process_0/customers.pkl')
with open('process_0/customer_map.pkl','wb') as f:
    pickle.dump(customer_map,f)

## customers.csv Done!

## 3.clean data of transactions_train.csv

In [ ]:
# read transactions_train.csv
transactions = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
print('transactions shape:',transactions.shape)
transactions['customer_id'] = transactions['customer_id'].map(lambda x: customer_map[x])
transactions['article_id'] = transactions['article_id'].map(lambda x: article_map[x])
transactions.head()

In [ ]:
if not os.path.exists('process_0'):
    os.mkdir('process_0')
pd.to_pickle(transactions,'process_0/transactions.pkl')

In [ ]:
os.listdir('process_0')

### 由于时间是从2018.9.20-2020.9.22 任务需求预测2020.9.23~9.29的购买。
### 考虑先利用2018.9.20-2019.8.30 作为训练集 然后2019.9.1~9.31 作为测试集 来测试模型的可行性。
### 最终在通过训练全量的数据集最小化mse，然后生成最终的结果。

In [ ]:
transactions.head()

## transactions_train.csv Done!

## 观察数据 是否有季节影响

In [ ]:
article_code.head()

In [ ]:
time_dict = {}
for dat in transactions.t_dat.unique():
    time_dict[dat] = int(dat[5:7])
t_dat_map = transactions.t_dat.copy()
t_dat_map = t_dat_map.map(lambda x: time_dict[x])

product_type_uni = article_code.product_type_no.unique()
product_group_uni = article_code.product_group_name.unique()
product_name_uni = article.prod_name.unique()

article2type = {}
for i, key in enumerate(article_code.article_id):
    article2type[key] = article_code.product_type_no[i]
product_type_size = len(article_code.product_type_no.unique())

article2group = {}
for i, key in enumerate(article_code.article_id):
    article2group[key] = article_code.product_group_name[i]
product_group_size = len(article_code.product_group_name.unique())

article2name = {}
for i, key in enumerate(article_code.article_id):
    article2name[key] = article.prod_name[i]
prod_name_size = len(article.prod_name.unique())

In [ ]:
def get_month_data(data, month, t_dat_map = t_dat_map):
    month_data = data.article_id[t_dat_map == month]
    return month_data
def give_month_get_res(month_data, product_uni = product_type_uni, dict = article2type):
    month_data = month_data.map(lambda x: dict[x])
    res = []
    for i in product_uni:
        res.append( (month_data==i).sum() )
    return res

def new_give_month_get_res(month_data, product_uni = product_type_uni, dict = article2type):
    month_data = month_data.map(lambda x: dict[x])
    res = []
    v_counts = month_data.value_counts()
    for i in product_uni:
        if i in v_counts.index:
            res.append( v_counts[i] )
        else:
            res.append( 0 )
    return res

#month_tmp = get_month_data(transactions, 1)
#res_tmp = give_month_get_res(month_tmp)

## 经观察发现 prod_name 即商品名公用45875种，如果只用product_type_name(132种)来衡量商品的话就显得不太充分。
### Notes: product_type_name = 'Dress'， 共有5165种prod_name与之相关联。

In [ ]:
import time
fig, axs = plt.subplots(3, 4, figsize=(36,12))
for i in range(12):
    row, col = int(i/4), i%4
    month_data = get_month_data(transactions, i+1)
    res = new_give_month_get_res(month_data, product_uni = product_name_uni, dict = article2name)
    axs[row, col].bar(list(range(40000)), height = res[:40000])
    axs[row, col].set_title(f"Month: {i+1} for prod name")
    print(f'fig: {i} done at {time.asctime()}')
fig.tight_layout()
plt.show()

## As we can see, distribution of about prod_name in month are many different!

In [ ]:
import matplotlib.pyplot as plt
fig, axs = plt.subplots(3, 4, figsize=(36,12))
for i in range(12):
    row, col = int(i/4), i%4
    month_data = get_month_data(transactions, i+1)
    res = new_give_month_get_res(month_data)
    axs[row, col].bar(product_type_uni[:80], height = res[:80])
    axs[row, col].set_title(f"Month: {i+1} for product type")
fig.tight_layout()
plt.show()

## As we can see, distribution of month 9, 10, 11 are more similar

In [ ]:
fig, axs = plt.subplots(3, 4, figsize=(36,12))
for i in range(12):
    row, col = int(i/4), i%4
    month_data = get_month_data(transactions, i+1)
    res = give_month_get_res(month_data, product_uni = product_group_uni, dict = article2group)
    axs[row, col].bar(product_group_uni, height = res)
    axs[row, col].set_title(f"Month: {i+1} for product group")
fig.tight_layout()
plt.show()

## can't find any userful information about product group name

## 用户会重复购买吗？先观察下用户的消费情况！

In [ ]:
user_trans = transactions.groupby('customer_id')

In [ ]:
repeat_buy_num = []
total_buy_num = []
for name, g in user_trans:
    total_buy_num.append(g.shape[0])
    tmp = pd.value_counts(g.article_id)
    repeat_buy_num.append(tmp.index[tmp>1].to_list())

In [ ]:
with open('process_0/repeat_buy_num.pkl','wb') as f:
    pickle.dump(repeat_buy_num,f)
with open('process_0/total_buy_num.pkl','wb') as f:
    pickle.dump(total_buy_num,f)

'''
with open('process_0/repeat_buy_num.pkl','rb') as f:
    tmp = pickle.load(f)
'''

In [ ]:
total_buy_pd = pd.Series(total_buy_num)

## there are 131514 个user only buy once
## 31788324 条购买记录中有 3515413条重复购买记录 （11%）

In [ ]:
(total_buy_pd == 1).sum()

In [ ]:
total_buy_pd.sum()

In [ ]:
type_count = [0] * len(article_code.product_type_no.unique())
len(type_count)

In [ ]:
type_count = [0] * len(article_code.product_type_no.unique())
for i in repeat_buy_num:
    for article_id in i:
        #print(f'article_id: {article_id} type_no: {article2type[article_id]}')
        type_count[article2type[article_id]] += 1

In [ ]:
for i,type in enumerate(type_count):
    print( i, type)
    if i == 80: break

In [ ]:
plt.bar(product_type_uni[:80], height = type_count[:80])
plt.show()

In [ ]:
article.head()

In [ ]:
article[article.product_type_name == 'Dress'].prod_name.unique().shape

In [ ]:
article.prod_name.unique().shape